Linear Model - Kiva

In [63]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [64]:
from Ingestion.kivadataloader import KivaDataLoader
import pandas as pd

m = KivaDataLoader()
cleaneduploans = m.get_clean_dataframe()

Connected to kivateam.cnd5idfvxmld.us-east-1.rds.amazonaws.com:5432/kiva
The process takes about 5 minutes to run.


In [65]:
cleaneduploans.head()

,loan_id,loan_name,language_english,description_length,funded_amount,loan_amount,status_loans,loan_image_provided,loan_video_provided,activity_name,...,sector_name_Manufacturing,sector_name_Personal Use,sector_name_Retail,sector_name_Services,sector_name_Transportation,sector_name_Wholesale,distribution_model_field_partner,repayment_interval_bullet,repayment_interval_irregular,repayment_interval_monthly
0,344960,Porn,1,539.0,1000.0,1000.0,funded,1,0,Vehicle,...,0,1,0,0,0,0,1,0,0,1
1,201245,Miriam del Socorro,0,858.0,300.0,300.0,funded,1,0,Tailoring,...,0,0,0,1,0,0,1,0,0,1
2,1351226,EVANS,1,506.0,500.0,500.0,funded,1,0,Blacksmith,...,1,0,0,0,0,0,1,0,1,0
3,1345013,San Miguel Group,0,706.0,5675.0,5675.0,funded,1,0,Agriculture,...,0,0,0,0,0,0,1,0,1,0
4,811265,Yaritza Stefany,0,675.0,450.0,450.0,funded,1,0,Food Stall,...,0,0,0,0,0,0,1,1,0,0


In [66]:
X = cleaneduploans.drop(['posted_to_raised_days', 'posted_to_raised_bins', 'disburse_to_posted_days','disbursed_to_raised_days','partner_id','funded_amount'], axis=1)
y = cleaneduploans['posted_to_raised_days']

In [67]:
X = X.select_dtypes(exclude=['object','datetime64[ns, UTC]'])

In [68]:
X.dropna(axis=1, inplace=True)

In [69]:
X.shape

(1170548, 38)

In [70]:
X.describe()

,loan_id,language_english,loan_amount,loan_image_provided,loan_video_provided,currency_usd,num_journal_entries,num_bulk_entries,tags_exist,atleast1_borrower_pictured,...,sector_name_Manufacturing,sector_name_Personal Use,sector_name_Retail,sector_name_Services,sector_name_Transportation,sector_name_Wholesale,distribution_model_field_partner,repayment_interval_bullet,repayment_interval_irregular,repayment_interval_monthly
count,1.170548e+06,1.170548e+06,1.170548e+06,1170548.0,1.170548e+06,1.170548e+06,1.170548e+06,1.170548e+06,1.170548e+06,1170548.0,...,1.170548e+06,1.170548e+06,1.170548e+06,1.170548e+06,1.170548e+06,1.170548e+06,1170548.0,1.170548e+06,1.170548e+06,1.170548e+06
mean,7.045821e+05,6.886971e-01,7.549698e+02,1.0,4.032299e-04,1.555049e-01,1.499342e+00,1.125682e+00,4.074758e-01,1.0,...,1.289738e-02,3.849735e-02,2.001703e-01,6.985104e-02,2.571787e-02,1.587291e-03,1.0,6.880196e-02,4.130074e-01,5.181906e-01
std,4.220334e+05,4.630266e-01,9.800240e+02,0.0,2.007655e-02,3.623855e-01,9.714595e-01,4.827181e-01,4.913649e-01,0.0,...,1.128319e-01,1.923937e-01,4.001279e-01,2.548959e-01,1.582924e-01,3.980920e-02,0.0,2.531172e-01,4.923743e-01,4.996692e-01
min,3.080000e+02,0.000000e+00,2.500000e+01,1.0,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.300058e+05,0.000000e+00,2.750000e+02,1.0,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.848595e+05,1.000000e+00,4.750000e+02,1.0,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.066568e+06,1.000000e+00,9.000000e+02,1.0,0.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000e+00,1.000000e+00,1.000000e+00
max,1.444068e+06,1.000000e+00,1.000000e+05,1.0,1.000000e+00,1.000000e+00,4.800000e+01,2.400000e+01,1.000000e+00,1.0,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.0,1.000000e+00,1.000000e+00,1.000000e+00


In [71]:
reg=linear_model.LinearRegression()

In [72]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=4)

In [73]:
reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [74]:
reg.coef_

array([ 3.90519189e-07, -9.88125752e-01,  1.15306286e-03, -3.78530582e-10,
       -8.39435050e-01,  2.55322960e-01,  1.70744699e-01, -1.07823442e-01,
        4.85639541e+00, -5.54029711e-10, -2.62397310e-08,  1.02577538e-05,
        1.03803507e+00,  6.48400811e-03,  8.24471452e-01, -1.18181530e-01,
        1.19664474e-01,  1.48294419e-03, -2.14715816e+00,  8.87303934e-01,
       -1.86916577e+00,  2.06835379e+00,  5.24368089e-01, -3.18037325e+00,
       -1.03162328e+00,  1.19704212e+00, -1.72492422e+00,  1.69919999e+00,
       -2.13798618e+00, -1.45460036e+00,  2.12847361e+00,  8.58815668e-01,
        2.03668056e+00, -1.56470005e-03,  0.00000000e+00,  4.38673417e-01,
       -1.13006928e+00,  6.91395865e-01])

In [75]:
a=reg.predict(X_test)

In [76]:
a

array([14.64632801,  8.40744695,  4.96265369, ..., 16.1861447 ,
       12.96085228,  7.48700537])

In [77]:
a[0]

14.646328010297053

In [78]:
y_test

25683      28.0
186166      7.0
309642      2.0
437042      3.0
807102      6.0
639155     13.0
451850     15.0
66973       5.0
613963      0.0
552619     12.0
107425      5.0
645756      6.0
527223      2.0
756642      7.0
671336      3.0
900358     15.0
542591     14.0
682522      6.0
278681     19.0
642968      6.0
887440      2.0
577694      9.0
761776     11.0
56787      10.0
238518      1.0
614165      1.0
29711      17.0
717879      5.0
709642      7.0
165761      8.0
           ... 
926995      4.0
28633       0.0
1162963     0.0
613409      5.0
265735     30.0
1082245    11.0
518509     12.0
661773     13.0
206756      1.0
1135652     4.0
1019993    20.0
1116603     0.0
1044649     8.0
907053     11.0
1067549    27.0
723745     30.0
1152669    26.0
683280      5.0
288393     23.0
451973     23.0
309694      0.0
210837      2.0
753737     10.0
707387      6.0
323409      0.0
140314      3.0
1131803     2.0
1121932    24.0
323729      5.0
406086     12.0
Name: posted_to_raised_d

In [79]:
#mean square error
np.mean((a-y_test)**2)

56.57005788348935

In [82]:
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     posted_to_raised_days   R-squared:                       0.156
Model:                               OLS   Adj. R-squared:                  0.156
Method:                    Least Squares   F-statistic:                     6783.
Date:                   Fri, 07 Dec 2018   Prob (F-statistic):               0.00
Time:                           19:51:07   Log-Likelihood:            -4.0207e+06
No. Observations:                1170548   AIC:                         8.041e+06
Df Residuals:                    1170515   BIC:                         8.042e+06
Df Model:                             32                                         
Covariance Type:               nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
loan_id                            3.916e-07   2.49e-08     15.728      0.000    3.43e-07     4.4e-07
language_english                     -1.0010      0.017    -58.990      0.000      -1.034      -0.968
loan_amount                           0.0011   9.21e-06    122.916      0.000       0.001       0.001
loan_image_provided                   1.5943      0.025     65.012      0.000       1.546       1.642
loan_video_provided                  -0.4383      0.347     -1.265      0.206      -1.118       0.241
currency_usd                          0.2546      0.020     12.436      0.000       0.214       0.295
num_journal_entries                   0.1751      0.015     11.885      0.000       0.146       0.204
num_bulk_entries                     -0.1317      0.029     -4.502      0.000      -0.189      -0.074
tags_exist                            4.8660      0.020    240.414      0.000       4.826       4.906
atleast1_borrower_pictured            1.5943      0.025     65.012      0.000       1.546       1.642
partner_total_amount_raised       -2.602e-08   1.02e-09    -25.581      0.000    -2.8e-08    -2.4e-08
partner_loans_posted               1.024e-05   3.69e-07     27.723      0.000    9.52e-06     1.1e-05
partner_charges_fees_and_interest     0.9907      0.071     14.034      0.000       0.852       1.129
partner_loans_at_risk_rate            0.0061      0.000     21.340      0.000       0.006       0.007
partner_url_length                    0.8349      0.023     36.072      0.000       0.790       0.880
num_borrowers_female                 -0.1205      0.004    -29.898      0.000      -0.128      -0.113
num_borrowers_male                    0.1259      0.006     19.728      0.000       0.113       0.138
num_borrowers                         0.0054      0.003      1.806      0.071      -0.000       0.011
num_borrowers_female_pct             -2.1542      0.023    -92.777      0.000      -2.200      -2.109
sector_name_Agriculture               0.9893      0.024     41.665      0.000       0.943       1.036
sector_name_Arts                     -1.8011      0.048    -37.837      0.000      -1.894      -1.708
sector_name_Clothing                  2.1705      0.034     63.842      0.000       2.104       2.237
sector_name_Construction              0.6190      0.059     10.576      0.000       0.504       0.734
sector_name_Education                -3.0840      0.040    -76.972      0.000      -3.163      -3.005
sector_name_Entertainment            -0.8557      0.170     -5.032      0.000      -1.189      -0.522
sector_name_Food                      1.2923      0.024     54.289      0.000       1.246       1.339
sector_name_Health                   -1.6259      0.062    -26.337      0.000      -1.747   